In [3]:
import numpy as np
from scipy.stats import qmc

import numpy as np

def compute_dipole_field(L, x_pos, y_pos, V, grid_x, grid_y):
    """
    Compute dipole electric field on a 2D grid
    
    Parameters:
    -----------
    L : float
        Antenna length (meters)
    x_pos : float
        X-position of antenna center
    y_pos : float
        Y-position of antenna center
    V : float
        Applied voltage (volts)
    grid_x : ndarray (2D)
        X-coordinates of grid points (from meshgrid)
    grid_y : ndarray (2D)
        Y-coordinates of grid points (from meshgrid)
    
    Returns:
    --------
    Ex_total : ndarray (2D)
        X-component of electric field at each grid point
    Ey_total : ndarray (2D)
        Y-component of electric field at each grid point
    """

    # 1. CHARGE MAGNITUDE
    # Simplified model: Q ∝ V × L
    Q = V * L * 1e-11  # Changed from 1e-9 to get more reasonable field values
    
    # 2. CHARGE POSITIONS
    x_plus = x_pos         # X-position of positive charge
    y_plus = y_pos + L/2   # Y-position of positive charge (top)
    
    x_minus = x_pos        # X-position of negative charge
    y_minus = y_pos - L/2  # Y-position of negative charge (bottom)
    
    # 3. COULOMB'S CONSTANT
    k = 8.99e9  # More accurate value: N⋅m²/C²
    
    # 4. FIELD FROM POSITIVE CHARGE (+Q)
    # Vector from charge to grid points
    dx_plus = grid_x - x_plus  # Shape: (grid_size, grid_size)
    dy_plus = grid_y - y_plus
    
    # Distance from positive charge to each grid point
    r_plus = np.sqrt(dx_plus**2 + dy_plus**2)
    
    # Avoid division by zero (minimum distance threshold)
    r_plus = np.maximum(r_plus, 1e-6)  # Clamp to minimum value
    
    # Electric field components from positive charge
    # E = k*Q*(r - r_charge)/|r - r_charge|³
    Ex_plus = k * Q * dx_plus / (r_plus**3)
    Ey_plus = k * Q * dy_plus / (r_plus**3)
    
    # 5. FIELD FROM NEGATIVE CHARGE (-Q)
    # Vector from charge to grid points
    dx_minus = grid_x - x_minus
    dy_minus = grid_y - y_minus
    
    # Distance from negative charge to each grid point
    r_minus = np.sqrt(dx_minus**2 + dy_minus**2)
    r_minus = np.maximum(r_minus, 1e-6)
    
    # Electric field components from negative charge
    # Note: Charge is -Q (negative)
    Ex_minus = k * (-Q) * dx_minus / (r_minus**3)
    Ey_minus = k * (-Q) * dy_minus / (r_minus**3)
    
    # 6. SUPERPOSITION: Total field is sum of both
    Ex_total = Ex_plus + Ex_minus
    Ey_total = Ey_plus + Ey_minus
    
    return Ex_total, Ey_total

def generate_dipole_dataset(
    n_circuits=100,
    grid_size=32,
    grid_range=0.3,
    l_bounds=(0.01, -0.1, -0.1, 1),
    u_bounds=(0.2,  0.1,  0.1, 100),
    field_clip=None,
    seed=42
):
    """
    Generate synthetic dipole dataset.

    Returns
    -------
    inputs  : (N_total, 6)
              [L, x_pos, y_pos, V, x, y]
    targets : (N_total, 2)
              [Ex, Ey]
    """

    np.random.seed(seed)

    # ======================================================
    # 1️⃣ Create spatial grid
    # ======================================================
    x = np.linspace(-grid_range, grid_range, grid_size)
    y = np.linspace(-grid_range, grid_range, grid_size)
    grid_x, grid_y = np.meshgrid(x, y)

    x_flat = grid_x.flatten()
    y_flat = grid_y.flatten()

    n_points = grid_size * grid_size

    # ======================================================
    # 2️⃣ Sample dipole parameters (Latin Hypercube)
    # ======================================================
    sampler = qmc.LatinHypercube(d=4, seed=seed)
    sample = sampler.random(n=n_circuits)
    sample = qmc.scale(sample, l_bounds, u_bounds)

    # ======================================================
    # 3️⃣ Compute fields
    # ======================================================
    results = []

    for L, x_pos, y_pos, V in sample:
        Ex, Ey = compute_dipole_field(L, x_pos, y_pos, V, grid_x, grid_y)
        results.append(np.stack([Ex, Ey], axis=-1))  # (grid, grid, 2)

    results_array = np.array(results)  # (n_circuits, grid, grid, 2)

    # Flatten fields
    results_array = results_array.reshape(-1, 2)  # (N_total, 2)

    # ======================================================
    # 4️⃣ Expand parameter inputs
    # ======================================================
    sample_extended = np.repeat(sample, repeats=n_points, axis=0)

    x_grid_extended = np.tile(x_flat, reps=n_circuits).reshape(-1, 1)
    y_grid_extended = np.tile(y_flat, reps=n_circuits).reshape(-1, 1)

    inputs = np.concatenate(
        [sample_extended, x_grid_extended, y_grid_extended],
        axis=1
    )

    targets = results_array

    # ======================================================
    # 5️⃣ Optional field clipping (for extreme outliers)
    # ======================================================
    if field_clip is not None:
        mask = (
            (np.abs(targets[:, 0]) <= field_clip) &
            (np.abs(targets[:, 1]) <= field_clip)
        )
        inputs = inputs[mask]
        targets = targets[mask]

    return inputs, targets


In [4]:
inputs, targets = generate_dipole_dataset(
    n_circuits=200,
    grid_size=64,
    grid_range=0.4,
    field_clip=3e4
)

print(inputs.shape)
print(targets.shape)


(819093, 6)
(819093, 2)


In [10]:
import torch.nn as nn

class SurrogateModel(nn.Module):
    def __init__(self,  hidden_dim=128, n_layers=3, input_dim = 6, output_dim = 2):

        super().__init__()

        layers_list = []
        for i in range(n_layers):
            layers_list.append(nn.Linear(hidden_dim, hidden_dim))
            layers_list.append(nn.ReLU())
            
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            *[layer for layer in layers_list],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)

In [11]:
# Loading the model and scalers
import torch
import joblib

X_scaler = joblib.load('../models/x_scaler.save')
y_scaler = joblib.load('../models/y_scaler.save')

model = SurrogateModel()
model.load_state_dict(torch.load('../models/dipole_surrogate_model.pth'))

model.eval()


SurrogateModel(
  (model): Sequential(
    (0): Linear(in_features=6, out_features=128, bias=True)
    (1): Linear(in_features=128, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
    (5): Linear(in_features=128, out_features=128, bias=True)
    (6): ReLU()
    (7): Linear(in_features=128, out_features=2, bias=True)
  )
)